In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.pandas.set_option('display.max_columns',None)


In [ ]:
dataset=pd.read_csv('fifa.csv')
var_info = pd.read_csv('fifa_+variable_information.csv')

# column: 'Value'
Details:
    Amount with Euro symbol as prefix and suffix ‘K’ or ‘M’ indicating thousands and millions respectively.
    
    
Required output:
        Convert to Float after getting rid of currency symbol and suffix.

In [ ]:
dataset['Value'] = dataset['Value'].apply(lambda x : x.split(',')[0][1:]) 

In [ ]:
def value_conversion(x):
    if(x[-1]=='M'):
        return float(x[0:-1])*1000000
    elif(x[-1]=='K'):
        return float(x[0:-1])*1000
    else:
        return float(x)


In [ ]:
dataset['Value'] = dataset['Value'].map(value_conversion)

In [ ]:
dataset['Value'].dtype

In [ ]:
dataset['Value'].isnull().sum()

# column: ''Wage''
Details:
    Amount with Euro symbol as prefix and suffix ‘K’ or ‘M’ indicating thousands and millions respectively.
    
    
Required output:
        Convert to Float after getting rid of currency symbol and suffix..

In [ ]:
dataset['Wage'] = dataset['Wage'].apply(lambda x : x.split(',')[0][1:]) 
dataset['Wage'] = dataset['Wage'].map(value_conversion)

In [ ]:
dataset['Wage'].isnull().sum()

# column: 'Joined'
Details: Year as a string, in some cases complete date as string

Required output: Convert to int with only year

In [ ]:
import datetime
dataset['Joined'] = pd.DatetimeIndex(dataset['Joined']).year
dataset['Joined'] = dataset['Joined'].fillna(dataset['Joined'].mean()).astype(int)

In [ ]:
len(dataset[dataset['Joined'].isnull()]['Joined'])

# column: 'Contract Valid Until'
Details: Date as a string 

Required output: Convert to datetime type

In [ ]:
dataset['Contract Valid Until'] = pd.to_datetime(dataset['Contract Valid Until'])
dataset['Contract Valid Until'] = pd.to_datetime(dataset['Contract Valid Until'],format='%Y%m%d')

# column: ''Height''
Details: In inches with a quotation mark  

Required output :Convert to Float with decimal points

In [ ]:
dataset['Height'] = dataset['Height'].str.replace("'",'.',regex=False)
dataset['Height'] = dataset['Height'].astype('float64')
dataset['Height'] = dataset['Height'].fillna(dataset['Height'].mean())

# column: 'Weight'
Details: Contains the suffix lbs

Required output :Remove the suffix and convert to float

In [ ]:
dataset['Weight'] = dataset['Weight'].str.replace("lbs",'',regex=False)
dataset['Weight'] = dataset['Weight'].astype('float64')
dataset['Weight'] = dataset['Weight'].fillna(dataset['Weight'].mean())

# column: 'Release Clause'
Details: Amount with Euro symbol as prefix and suffix ‘K’ or ‘M’ indicating thousands and millions respectively

Required output :Convert to Float after getting rid of currency symbol and suffix

In [ ]:
dataset['Release Clause'] = dataset['Release Clause'].fillna('E0')
dataset['Release Clause']=dataset['Release Clause'].apply(lambda x : x[1:])
dataset['Release Clause'] = dataset['Release Clause'].map(value_conversion)
dataset['Release Clause'] = dataset['Release Clause'].replace(0,np.nan)
dataset['Release Clause']= dataset['Release Clause'].fillna(dataset['Release Clause'].mean())

In [ ]:
#####################################################################################3

In [ ]:
dataset.head()

# 4.	Check for missing values and do a mean imputation where necessary.

In [ ]:
features_with_nan = [feature for feature in dataset.columns if dataset[feature].isnull().sum()>1]

In [ ]:
for feature in features_with_nan:
    print('{} missing values are ==========>>>{}'.format(feature,np.round(dataset[feature].isnull().mean(),4)))

In [ ]:
numerical_features = [feature for feature in dataset.columns if dataset[feature].dtype != 'O']
numerical_features

In [ ]:
dataset['Loaned From'].isnull().sum()

In [ ]:
# since Loaned From contains many(almose 90%) null values ..we can remove the column 

In [ ]:
dataset.drop(columns='Loaned From',inplace=True)

In [ ]:
dataset[['Overall','Potential']].sample(10)

In [ ]:
#since the dataset contains same values for both the columns overall and potential we can delete any one of the column

In [ ]:
#dataset.drop(columns=['Potential'],inplace=True)

In [ ]:
dataset[dataset['Name'].duplicated()]['Name']

In [ ]:
#the name column has duplicates..there are players with same name

In [ ]:
dataset.head()

In [ ]:
len(dataset['Flag'].unique())

In [ ]:
dataset['Body Type'].sample(10)

In [ ]:
#photo,flag and club logo,Jersey Number columns can be deleted because we have the details like id,name,nationality and club name to represent them

In [ ]:
dataset.drop(columns=['Photo','Flag','Club Logo','Jersey Number'],inplace=True)

In [ ]:
dataset.head()

In [ ]:
#now for numerical columns we replace the null values with mean
#if there are any outliers then we replace with median


In [ ]:
numerical_features = [feature for feature in dataset.columns if dataset[feature].dtype != 'O']
for feature in numerical_features:
    dataset[feature] = dataset[feature].fillna(dataset[feature].mean())

In [ ]:
#after applying mean to fill nulls..now checking the null values for numerical_columns:
for feature in numerical_features:
    print(dataset[feature].isnull().sum())

In [ ]:
#now checking the null values for categorical columns

In [ ]:
categorical_features=[feature for feature in dataset.columns if dataset[feature].dtypes=='O']
for feature in categorical_features:
    print('{} => missing values are {}'.format(feature,dataset[feature].isnull().sum()))

In [ ]:
#we are replacing these null values with "missing" string

In [ ]:
for feature in categorical_features:
    dataset[feature] = dataset[feature].fillna('missing')

In [ ]:
#now our data is complete.all null values are treated.
dataset.isnull().sum()

In [ ]:
dataset.to_csv('cleaned_minipro.csv')

# EDA PART

# 1.	Plot the distribution of Overall rating for all players. 

In [ ]:
from scipy.stats import norm,skew
sns.distplot(dataset['Overall'])
plt.axvline(dataset['Overall'].mean(),label = 'mean',color = 'red')
plt.axvline(dataset['Overall'].median(),label = 'median',color = 'blue')
plt.axvline(dataset['Overall'].mode()[0],label = 'mode',color = 'green') # ---> here we get series for mode hence using [0] to get value
plt.legend()
plt.show()

In [ ]:
#we can clearly see the Potential is almost normal(evenly distributed)
#the mean and median is around 67


# 2.	Generate pair plots for the following variables:
Overall, Value, Wage, International Reputation, Height, Weight, Release Clause


In [ ]:
pplot_features = dataset[['Overall','Wage','International Reputation','Height','Weight','Release Clause']]

In [ ]:
pplot_features.corr()

In [ ]:
sns.pairplot(pplot_features.corr())
plt.show()
#fig = sns.pairplot(pplot_features.corr()).get_figure()

#sns.pairplot(pplot_features.corr()).savefig('pairplot.png')

In [ ]:
#(wage -overall),(international reputation-overall),(release clause - overall) ===>> these have a positive correlation
#(wage-internaltional repuation),(release clause-wage)==> has a positive correlation

# 3.	Generate a table containing the top 20 players ranked by Overall score and whose contract expires in 2020.
    a)	What would the average wage for this set of players be?
    b)	What is the average age?
    c)	Is there a correlation between the Overall rating and Value for these players?



In [ ]:
dataset1 = dataset[dataset['Contract Valid Until'].dt.year == 2020]
#dataset1['Overall_dense_rank'] = dataset1['Overall'].rank(method='dense',ascending=False)

In [ ]:
req_table = dataset1.sort_values(by='Overall',ascending=False)[:20]
print('The average wage of required players is :',req_table['Wage'].mean())
print('The average age of required players is :',req_table['Age'].mean())

In [ ]:
sns.heatmap(dataset1[['Overall','Value']].corr(),annot=True)
plt.show()

In [ ]:
#yes,there is a positive correlation between the overall rating and value of the player

# 4.	Generate tables containing the top 5 players by Overall rating for each unique position.
    a)	Are there any players appearing in more than one Table. Please point out such players.
    b)	What is the average wage one can expect to pay for the top 5 in every position?



In [ ]:
position_features = dataset['Position'].unique()
position_features
position_features1=position_features
print(position_features1)
print()

In [ ]:
for i in range(0,len(position_features)):
    position_features1[i] = dataset[dataset['Position'] == position_features[i]].sort_values('Overall',ascending =False)[:5]

In [ ]:
result = pd.concat(position_features1)

In [ ]:
result.head(10)

In [ ]:
result[result.duplicated(subset=['ID'])]

In [ ]:
# There are no players who appeared more than once in top 5 as per their positions

In [ ]:
avg_wage_for_position = result.groupby('Position')['Wage'].mean()

In [ ]:
avg_wage_for_position = pd.DataFrame(avg_wage_for_position)

In [ ]:
#avg_wage_for_position

In [ ]:
avg_wage_for_position.drop('missing',axis=0,inplace=True)

In [ ]:
avg_wage_for_position.sort_values(by='Wage',ascending=False).plot(kind='bar')
plt.show()

In [ ]:
#The average wages as per position is plotted using bar graph
#ST position has the highest avg wage
#LWB position has the lowest avg.wage